### Fully Connected Neural Network - Multivariate Regression

In [319]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import pandas as pd
import requests
import collections

full_data = pd.read_csv('f1_data.csv')

##### Common Helper Functions

In [320]:
# Using the formuladata api

def get_location_details(location):
    url = 'https://formuladataapi.pythonanywhere.com/api/f1/circuit_data'
    filters = {}
    filters['location'] = location
    response = requests.get(url, params=filters)
    data = response.json()
    try:
      latitude = data[0]['latitude']
      longitude = data[0]['longitude']
      circuit_length = float(data[0]['circuit_length'][0:3])
    except:
      return None
    return [latitude, longitude, circuit_length]


def get_fp_details(driver, season, round):
    url = 'https://formuladataapi.pythonanywhere.com/api/f1'
    filters = {}
    filters['driver_name'] = driver
    filters['round'] = round
    filters['season'] = season
    response = requests.get(url, params=filters)
    data = response.json()
    try:
      fp1 = int(data[0]['fp1_position'])
    except:
      fp1 = None
    try:
      fp2 = int(data[0]['fp2_position'])
    except:
      fp2 = None
    try:
      fp3 = int(data[0]['fp3_position'])
    except:
      fp3 = None
    return [fp1, fp2, fp3]
       

In [321]:
def get_race_results_with_fp(map, season, round, location, weather, XX, model, fps):
    race_results = {}
    weather_dict = {'dry':0, "cloudy":1, "wet":2}
    latitude = get_location_details(location)[0]
    longitude = get_location_details(location)[1]
    circuit_length = get_location_details(location)[2]

    for driver, team in map.items():
        datapoint = [0]*XX.shape[1]
        datapoint[0] = season
        datapoint[1] = round
        datapoint[2] = weather_dict[weather]
        try:
            datapoint[3] = int(fps[driver][0])
        except:
            datapoint[3] = 20
        try:
            datapoint[4] = int(fps[driver][1])
        except:
            datapoint[4] = 20
        try:
            datapoint[5] = int(fps[driver][2])
        except:
            datapoint[5] = 20
        datapoint[6] = circuit_length
        datapoint[7] = latitude
        datapoint[8] = longitude

        loc = location.lower().replace(' ', '_')
        location_index = XX.columns.get_loc(f'location_{loc}')
        datapoint[location_index] = 1
    
        driver_index = XX.columns.get_loc(f'driver_name_{driver}')
        team_index = XX.columns.get_loc(f'constructor_name_{team}')
        datapoint[driver_index] = 1
        datapoint[team_index] = 1

        df = pd.DataFrame([datapoint], columns=XX.columns)
        test_prediction = model.predict(df ,verbose=0)
        race_results[driver] = test_prediction
    sorted_results = sorted(race_results.items(), key=lambda x: x[1])
    sorted_results = collections.OrderedDict(sorted_results)
    return sorted_results

In [322]:
def get_race_results(map, season, round, location, weather, XX, model):
    race_results = {}
    weather_dict = {'dry':0, "cloudy":1, "wet":2}
    latitude = get_location_details(location)[0]
    longitude = get_location_details(location)[1]
    circuit_length = get_location_details(location)[2]

    for driver, team in map.items():
        datapoint = [0]*X_test.shape[1]
        datapoint[0] = season
        datapoint[1] = round
        datapoint[2] = weather_dict[weather]

        datapoint[3] = circuit_length
        datapoint[4] = latitude
        datapoint[5] = longitude

        loc = location.lower().replace(' ', '_')
        location_index = X.columns.get_loc(f'location_{loc}')
        datapoint[location_index] = 1
    
        driver_index = X.columns.get_loc(f'driver_name_{driver}')
        team_index = X.columns.get_loc(f'constructor_name_{team}')
        datapoint[driver_index] = 1
        datapoint[team_index] = 1

        df = pd.DataFrame([datapoint], columns=XX.columns)
        test_prediction = model.predict(df ,verbose=0)
        race_results[driver] = test_prediction
    sorted_results = sorted(race_results.items(), key=lambda x: x[1])
    sorted_results = collections.OrderedDict(sorted_results)
    return sorted_results

In [323]:
driver_team_mapping = {
    'max_verstappen': 'red_bull_racing',
    'fernando_alonso': 'aston_martin',
    'lewis_hamilton': 'mercedes',
    'charles_leclerc': 'ferrari',
    'carlos_sainz': 'ferrari',
    'sergio_perez': 'red_bull_racing',
    'alexander_albon': 'williams',
    'esteban_ocon': 'aston_martin',
    'lance_stroll': 'aston_martin',
    'valtteri_bottas': 'alfa_romeo',
    'oscar_piastri': 'mclaren',
    'pierre_gasly': 'renault',
    'lando_norris': 'mclaren',
    'yuki_tsunoda': 'toro_rosso',
    'nico_hulkenberg': 'haas',
    'zhou_guanyu': 'alfa_romeo',
    'kevin_magnussen': 'haas',
    'nyck_de_vries': 'toro_rosso',
    'george_russell': 'mercedes',
    'logan_sargeant': 'williams'
}


In [324]:
def create_model(num_features, units=(128, 64), activation='relu', dropout_rate=0.0):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units[0], activation=activation, input_shape=(num_features,)),
        tf.keras.layers.Dense(units[1], activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


##### Predicting Race Positions

In [325]:
race_data = full_data.drop(["has_fastest_lap","race_laps_completed","points","grid_position", "fastest_lap_position"], axis = 1)
race_data = race_data.iloc[:]
# 2021: 14744

X = race_data.drop('race_finishing_position', axis=1)  
y = race_data['race_finishing_position']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
num_features = X_train.shape[1]
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (12643, 416) (12643,)
Testing data shape: (3161, 416) (3161,)


In [326]:
race_model = create_model(num_features,(64, 32), 'relu', 0.1)

In [327]:
# param_grid = {
#     'units': [(128, 64, 32, 16, 8), (128, 64, 32, 16, 8)],
#     'activation': ['relu', 'sigmoid'],
#     'dropout_rate': [0.0, 0.2, 0.4]
# }

# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
# grid_search.fit(X_train, y_train)

# print("Best Hyperparameters:", grid_search.best_params_)

# best_model = grid_search.best_estimator_
# best_model.fit(X_train, y_train)
# predictions = model.predict(X_test)

# mse = best_model.score(X_test, y_test)
# print("Mean Squared Error (MSE):", mse)

In [328]:
race_model.fit(X_train, y_train, epochs=100, batch_size=32)
loss = race_model.evaluate(X_test, y_test)
predictions = race_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error:', mse)

Epoch 1/100
396/396 [==============================] - 2s 2ms/step - loss: 111.8628
Epoch 2/100
396/396 [==============================] - 1s 2ms/step - loss: 43.3757
Epoch 3/100
396/396 [==============================] - 1s 2ms/step - loss: 39.7454
Epoch 4/100
396/396 [==============================] - 1s 3ms/step - loss: 39.4158
Epoch 5/100
396/396 [==============================] - 1s 2ms/step - loss: 38.9989
Epoch 6/100
396/396 [==============================] - 1s 2ms/step - loss: 38.8676
Epoch 7/100
396/396 [==============================] - 1s 2ms/step - loss: 38.6121
Epoch 8/100
396/396 [==============================] - 1s 2ms/step - loss: 38.0439
Epoch 9/100
396/396 [==============================] - 1s 2ms/step - loss: 38.2531
Epoch 10/100
396/396 [==============================] - 1s 2ms/step - loss: 37.1644
Epoch 11/100
396/396 [==============================] - 1s 2ms/step - loss: 37.3972
Epoch 12/100
396/396 [==============================] - 1s 3ms/step - loss: 37.9201


In [329]:
print(predictions[0:10])
print(y_test[0:10])

[[ 8.692207 ]
 [ 5.0198603]
 [ 4.850745 ]
 [ 9.438948 ]
 [11.12249  ]
 [14.874588 ]
 [16.936491 ]
 [10.842731 ]
 [17.527369 ]
 [ 4.529587 ]]
7841      2
13845     2
12462     1
15157    14
13993    10
11498    20
4363     20
718       6
11441    13
12971     6
Name: race_finishing_position, dtype: int64


In [330]:
fps = {}
for driver, _ in driver_team_mapping.items():
    drivers = driver.split('_')
    for d in range(len(drivers)):
        drivers[d] = drivers[d][0].upper() + drivers[d][1:]
    driver_parsed = ' '.join(drivers)
    fps[driver] = get_fp_details(driver_parsed, 2023, 10)

print(fps)

{'max_verstappen': [1, 1, 8], 'fernando_alonso': [4, 10, 3], 'lewis_hamilton': [12, 15, 5], 'charles_leclerc': [5, None, 1], 'carlos_sainz': [7, 2, 6], 'sergio_perez': [2, 4, 14], 'alexander_albon': [3, 3, 2], 'esteban_ocon': [6, 13, 16], 'lance_stroll': [9, 6, 13], 'valtteri_bottas': [15, 16, 18], 'oscar_piastri': [10, 9, 17], 'pierre_gasly': [13, 8, 4], 'lando_norris': [8, 14, 12], 'yuki_tsunoda': [16, 18, 10], 'nico_hulkenberg': [20, 7, 19], 'zhou_guanyu': [18, 11, 20], 'kevin_magnussen': [19, 17, 15], 'nyck_de_vries': [11, 19, 11], 'george_russell': [14, 12, 9], 'logan_sargeant': [17, 5, 7]}


In [331]:
results_race = get_race_results_with_fp(driver_team_mapping, 2023, 10, 'Great Britain', 'cloudy', X_test, race_model, fps)

In [332]:
arr_race = []

for key, val in results_race.items():
    arr_race.append((key,"{:.2f}".format(float(val))))


for item in arr_race:
    print(item[0], item[1])


max_verstappen 5.18
alexander_albon 5.98
sergio_perez 6.26
lewis_hamilton 6.82
carlos_sainz 7.27
fernando_alonso 7.33
charles_leclerc 9.31
pierre_gasly 9.50
george_russell 9.98
lando_norris 10.72
lance_stroll 11.15
logan_sargeant 11.70
esteban_ocon 12.06
oscar_piastri 13.70
valtteri_bottas 15.35
nico_hulkenberg 15.42
nyck_de_vries 15.56
yuki_tsunoda 15.74
kevin_magnussen 16.61
zhou_guanyu 16.63


##### Predicting Qualifying Positions


In [333]:

quali_data = full_data.drop(["race_finishing_position", "has_fastest_lap","race_laps_completed","points", "fastest_lap_position"], axis = 1)
quali_data = quali_data.iloc[:]
# 2021: 14744

X = quali_data.drop('grid_position', axis=1)  
y = quali_data['grid_position']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
num_features = X_train.shape[1]
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (12643, 416) (12643,)
Testing data shape: (3161, 416) (3161,)


In [334]:
quali_model = create_model(num_features,(64, 32), 'relu', 0.1)

In [335]:
quali_model.fit(X_train, y_train, epochs=100, batch_size=32)
loss = quali_model.evaluate(X_test, y_test)
predictions = quali_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error:', mse)

Epoch 1/100
396/396 [==============================] - 2s 2ms/step - loss: 158.2258
Epoch 2/100
396/396 [==============================] - 1s 2ms/step - loss: 25.8434
Epoch 3/100
396/396 [==============================] - 1s 2ms/step - loss: 22.0961
Epoch 4/100
396/396 [==============================] - 1s 2ms/step - loss: 21.2032
Epoch 5/100
396/396 [==============================] - 1s 4ms/step - loss: 20.0335
Epoch 6/100
396/396 [==============================] - 2s 4ms/step - loss: 20.1046
Epoch 7/100
396/396 [==============================] - 1s 3ms/step - loss: 20.1217
Epoch 8/100
396/396 [==============================] - 1s 3ms/step - loss: 19.6610
Epoch 9/100
396/396 [==============================] - 1s 3ms/step - loss: 19.3141
Epoch 10/100
396/396 [==============================] - 2s 4ms/step - loss: 19.0096
Epoch 11/100
396/396 [==============================] - 1s 3ms/step - loss: 18.8871
Epoch 12/100
396/396 [==============================] - 1s 4ms/step - loss: 19.4861


In [336]:
print(predictions[0:10])
print(y_test[0:10])

[[ 8.785857 ]
 [ 5.714333 ]
 [ 5.6842403]
 [10.526288 ]
 [11.707262 ]
 [13.42287  ]
 [19.176945 ]
 [ 8.790802 ]
 [19.811756 ]
 [ 4.64921  ]]
7841      7
13845     2
12462     2
15157     8
13993    15
11498    16
4363     14
718       1
11441    18
12971     4
Name: grid_position, dtype: int64


In [337]:
results_quali = get_race_results_with_fp(driver_team_mapping, 2023, 10, 'Great Britain', 'cloudy', X_test, quali_model, fps)

In [338]:
arr_quali = []
for key, val in results_quali.items():
    arr_quali.append((key,"{:.2f}".format(float(val))))
for item in arr_quali:
    print(item[0], item[1])


alexander_albon 5.12
max_verstappen 5.44
carlos_sainz 6.14
sergio_perez 8.35
charles_leclerc 8.57
fernando_alonso 8.64
lewis_hamilton 8.95
pierre_gasly 8.96
logan_sargeant 10.10
lando_norris 11.61
george_russell 11.66
lance_stroll 13.14
oscar_piastri 13.81
esteban_ocon 14.50
nico_hulkenberg 15.12
nyck_de_vries 15.49
yuki_tsunoda 16.27
valtteri_bottas 16.53
zhou_guanyu 16.82
kevin_magnussen 17.45


##### Predict Fastest Lap

In [339]:
fl_data = full_data.drop(["race_finishing_position", "has_fastest_lap","race_laps_completed","points", "grid_position"], axis = 1)
fl_data = fl_data.iloc[:]
# 2021: 14744

X = fl_data.drop('fastest_lap_position', axis=1)  
y = fl_data['fastest_lap_position']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
num_features = X_train.shape[1]
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (12643, 416) (12643,)
Testing data shape: (3161, 416) (3161,)


In [340]:
fl_model = create_model(num_features,(64, 32), 'relu', 0.1)
fl_model.fit(X_train, y_train, epochs=100, batch_size=32)
loss = fl_model.evaluate(X_test, y_test)
predictions = fl_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error:', mse)

Epoch 1/100
396/396 [==============================] - 2s 3ms/step - loss: 126.1641
Epoch 2/100
396/396 [==============================] - 1s 3ms/step - loss: 28.9262
Epoch 3/100
396/396 [==============================] - 2s 4ms/step - loss: 26.3465
Epoch 4/100
396/396 [==============================] - 1s 3ms/step - loss: 26.0518
Epoch 5/100
396/396 [==============================] - 2s 4ms/step - loss: 26.3635
Epoch 6/100
396/396 [==============================] - 2s 4ms/step - loss: 26.4032
Epoch 7/100
396/396 [==============================] - 2s 4ms/step - loss: 25.3730
Epoch 8/100
396/396 [==============================] - 2s 4ms/step - loss: 26.2225
Epoch 9/100
396/396 [==============================] - 1s 4ms/step - loss: 25.0539
Epoch 10/100
396/396 [==============================] - 2s 4ms/step - loss: 24.4500
Epoch 11/100
396/396 [==============================] - 2s 4ms/step - loss: 24.7185
Epoch 12/100
396/396 [==============================] - 1s 4ms/step - loss: 24.3389


In [341]:
print(predictions[0:10])
print(y_test[0:10])

[[ 8.091401]
 [ 4.660547]
 [ 4.544779]
 [ 9.962818]
 [10.386635]
 [11.750876]
 [15.57021 ]
 [ 7.018102]
 [19.22105 ]
 [ 4.190813]]
7841      3
13845     5
12462     1
15157     7
13993     6
11498    11
4363     12
718       6
11441    15
12971     5
Name: fastest_lap_position, dtype: int64


In [342]:
results_fl = get_race_results_with_fp(driver_team_mapping, 2023, 10, 'Great Britain', 'cloudy', X_test, fl_model, fps)

In [343]:
arr_fl = []
for key, val in results_fl.items():
    arr_fl.append((key,"{:.2f}".format(float(val))))
for item in arr_fl:
    print(item[0], item[1])

max_verstappen 4.02
alexander_albon 4.64
sergio_perez 5.24
carlos_sainz 6.41
lewis_hamilton 8.19
fernando_alonso 8.41
logan_sargeant 9.60
pierre_gasly 9.97
charles_leclerc 10.03
george_russell 10.58
lando_norris 10.87
lance_stroll 11.00
esteban_ocon 13.48
oscar_piastri 13.93
nico_hulkenberg 14.08
yuki_tsunoda 14.12
zhou_guanyu 14.75
nyck_de_vries 14.86
valtteri_bottas 15.30
kevin_magnussen 16.04


In [344]:
print(arr_race)
print("........................")
print(arr_quali)
print("........................")
print(arr_fl)

[('max_verstappen', '5.18'), ('alexander_albon', '5.98'), ('sergio_perez', '6.26'), ('lewis_hamilton', '6.82'), ('carlos_sainz', '7.27'), ('fernando_alonso', '7.33'), ('charles_leclerc', '9.31'), ('pierre_gasly', '9.50'), ('george_russell', '9.98'), ('lando_norris', '10.72'), ('lance_stroll', '11.15'), ('logan_sargeant', '11.70'), ('esteban_ocon', '12.06'), ('oscar_piastri', '13.70'), ('valtteri_bottas', '15.35'), ('nico_hulkenberg', '15.42'), ('nyck_de_vries', '15.56'), ('yuki_tsunoda', '15.74'), ('kevin_magnussen', '16.61'), ('zhou_guanyu', '16.63')]
........................
[('alexander_albon', '5.12'), ('max_verstappen', '5.44'), ('carlos_sainz', '6.14'), ('sergio_perez', '8.35'), ('charles_leclerc', '8.57'), ('fernando_alonso', '8.64'), ('lewis_hamilton', '8.95'), ('pierre_gasly', '8.96'), ('logan_sargeant', '10.10'), ('lando_norris', '11.61'), ('george_russell', '11.66'), ('lance_stroll', '13.14'), ('oscar_piastri', '13.81'), ('esteban_ocon', '14.50'), ('nico_hulkenberg', '15.12'